In [1]:
from agol_restapi_tools import token_generation
from agol_restapi_tools import agol_table_to_pd
from agol_restapi_tools import oid_field
from agol_restapi_tools import locate_objectid
from agol_restapi_tools import pd_to_attributes_list

import requests
import pandas as pd
import datetime
from pytz import timezone
import pytz
import json

import csv
from datetime import datetime, timedelta

#Survey URL
survey_url = 'https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/APEX_FORM_VIEW/FeatureServer'

#Starts URL
starts_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/Start_Points/FeatureServer"

#Ends URL
ends_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/End_Points/FeatureServer"

#Projects URL
projects_url= "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/CY2023_Project_Polygons_notcleaned/FeatureServer"

#Alert Logbook URL
alert_log_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/emailed_managers_log/FeatureServer"

#Update Logbook URL
update_log_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/update_log/FeatureServer"

#Alaska 511 Management URL
management_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/AK_511_Management/FeatureServer"

#Set Token
token = token_generation(username = "AKDOT_COMMUNICATIONS", password = 'Skisnowbird1')

In [2]:
projects = agol_table_to_pd(projects_url, 0, token)

In [3]:
projects = projects[projects['Unique_ID'] == 117]

In [4]:
points_url = "https://services.arcgis.com/r4A0V7UzH9fcLVvv/arcgis/rest/services/CY2023_Project_Points/FeatureServer"
points = agol_table_to_pd(points_url, 57, token)

In [5]:
#Add new project data one at a time
for index, row in projects.iterrows():
    lat = row['Latitude']
    lng = row['Longitude']

    data = pd.DataFrame(row).T

    data.drop(columns = ['Shape__Area', 'Shape__Length'], inplace = True)

    

    payload = pd_to_attributes_list(pd.DataFrame(row).T)
    
    for key, value in payload[0]['attributes'].items():
            if type(value) == str:
                if '\xa0' in value:
                    value = value.replace('\xa0', '')
                    payload[0]['attributes'][key] = value

                    
    url = f"{points_url}/57/applyEdits"

    params = {
        'f': 'json',
        'token': token,
        'adds':f'{payload}'
    }


    response = requests.post(url, params).json()

    objectid = response['addResults'][0]['objectId']

    geo_payload =  [
        {
        "geometry": {
            "x": lng,
            "y": lat,
            'spatialReference': {
                'wkid': 4326 }
        },
        "attributes": {
            'OBJECTID': objectid
        } 
        } 
            ]
    
    geo_params = {
                'f':'json',
                'token': token,
                'updates':f'{geo_payload}'
            }
    
    print(requests.post(url, geo_params).json())

{'addResults': [], 'updateResults': [{'objectId': 261, 'uniqueId': 261, 'globalId': None, 'success': True}], 'deleteResults': []}


In [ ]:
geo_payload =  [
        {
        "geometry": {
            "x": lng,
            "y": lat,
            # 'spatialReference': {
            #     'wkid': 4326 }
        },
        "attributes": {
            '': ''
        } 
        } 
            ]

In [89]:
points.columns

Index(['OBJECTID', 'Unique_ID', 'Project_Name', 'IRIS_Number',
       'Federal_Project_Number', 'STIP_ID', 'Route_ID', 'Route_Name',
       'Project_Description', 'Project_Purpose', 'Project_Website',
       'Project_Engineer_Email', 'Design_Engineer', 'Design_Engineer_Phone',
       'Construction_Engineer', 'Construction_Engineer_Phone',
       'Contract_Value', 'Anticipated_Start', 'Anticipated_End',
       'Project_Practice', 'Funding_Type', 'Project_Location', 'Project_Limit',
       'Project_Impact', 'Construction_Description', 'Start_Milepost',
       'End_Milepost', 'DOT_PF_Region', 'Borough_CensusArea',
       'State_House_District', 'State_Senate_District',
       'Project_Priority_Area', 'Longitude', 'Latitude', 'Video_URL',
       'Creation_Date', 'Creator', 'Edit_Date', 'Editor'],
      dtype='object')